In [28]:
import gurobipy as gp
from gurobipy import GRB

# a,b,c=(model.addVar(lb=0, vtype=GRB.INTEGER, name=f"popul_var{i+1}") for i in range(3))
# K,B,D=(model.addVar(lb=0, vtype=GRB.INTEGER, name=f"budget{i+1}") for i in range(3))
# A=[[model.addVar(lb=0, vtype=GRB.BINARY, name=f"a{i}_{j}") for i in range(9)] for j in range(9)]
# # Create a Gurobi model
# population=[model.addVar(lb=0, vtype=GRB.BINARY, name=f"populationR{i}") for i in range(9)]
population=[2,3,4,5,6,7,8,9,10]
population=[i*10 for i in population]
A=[
    [1,1,0,0,1,0,0,0,0],
    [1,1,1,1,1,0,0,0,0],
    [0,1,1,1,0,0,0,0,0],
    [0,1,1,1,1,0,1,0,0],
    [1,1,0,1,1,0,1,0,0],
    [0,0,0,0,0,1,1,0,1],
    [0,0,0,1,1,1,1,1,0],
    [0,0,0,0,0,0,1,1,1],
    [0,0,0,0,0,1,0,1,1]
    ]
K=500000
D=20000
B=2000000
a=5
b=1
c=2

model = gp.Model("linear_program")
agencies=[model.addVar(lb=0, vtype=GRB.BINARY, name=f"ag{i+1}") for i in range(9)]
Server_DAB=[model.addVar(lb=0, vtype=GRB.BINARY, name=f"DAB{i+1}") for i in range(9)]
Z=0
for i,(Ag,Se) in enumerate(zip(agencies,Server_DAB)):
    population_voisine=sum([j*pop for j,pop in zip(A[i],population)])-A[i][i]*population[i]
    Z+=population[i]*(a*Ag+c*Se)+population_voisine*(Ag*b)

model.setObjective(Z, sense=GRB.MAXIMIZE)
for i in range(9):
    for j in range(i+1,9):
        model.addConstr(agencies[i]+agencies[j]<=1)
model.addConstr(sum(agencies)*K+sum(Server_DAB)*D <=B)
model.optimize()
print("\nOptimal solution:")
for var in model.getVars():
    print(f"{var.varName}: {var.x}")
print(f"\nOptimal objective value: {model.objVal}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Core(TM) i5-8300H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 37 rows, 18 columns and 90 nonzeros
Model fingerprint: 0xc6748d1d
Variable types: 0 continuous, 18 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+05]
  Objective range  [4e+01, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+06]
Found heuristic solution: objective 1270.0000000
Presolve removed 37 rows and 18 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 1750 1270 



Optimal solution found (tolerance 1.00e-04)
Best objective 1.750000000000e+03, best bound 1.750000000000e+03, gap 0.0000%

Optimal solution:
ag1: 0.0
ag2: 0.0
ag3: 0.0
ag4: 0.0
ag5: 0.0
ag6: 0.0
ag7: 1.0
ag8: 0.0
ag9: 0.0
DAB1: 1.0
DAB2: 1.0
DAB3: 1.0
DAB4: 1.0
DAB5: 1.0
DAB6: 1.0
DAB7: 1.0
DAB8: 1.0
DAB9: 1.0

Optimal objective value: 1750.0


In [31]:
import gurobipy as gp
from gurobipy import GRB
model = gp.Model("linear_program")
place_antene=[model.addVar(lb=0, vtype=GRB.BINARY, name=f"{i}") for i in "ABCDEFG"]
place_antene_acces=[[1,2],[1,3],[1,4],[3,4],[2,4],[2,5],[4,5]]
model.setObjective(sum(place_antene), sense=GRB.MINIMIZE)
contraint=[1,1,1,2,1]
for i in range(1,5):
    Z=0
    for j in range(7):
        if i in place_antene_acces[j]:
            Z+=place_antene[j]

    model.addConstr(Z>=contraint[i])
model.optimize()
# Print the results
print("\nOptimal solution:")
for var in model.getVars():
    print(f"{var.varName}: {var.x}")

print(f"\nOptimal objective value: {model.objVal}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Core(TM) i5-8300H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 7 columns and 12 nonzeros
Model fingerprint: 0x048833b9
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 3.0000000
Presolve removed 4 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%

Optimal solution:
A: 0.0
B: 1.0
C: 0.0
D: 1.0
